In [1]:
import torch.nn as nn
import torch

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm

from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
from skimage.filters import threshold_local, gaussian
import ntpath

/home/harishbabu/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/harishbabu/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
# run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = '/home/harishbabu/projects/PIPNet/runs/032-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/035-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'

# run_path = '/home/harishbabu/projects/PIPNet/runs/043-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel'
# run_path = "/home/harishbabu/projects/PIPNet/runs/036-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel_uniformity"
# run_path = "/home/harishbabu/projects/PIPNet/runs/041-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/042-035_clone-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/044-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=20-or-4per-desc_orth-on-rel"

# run_path = "/home/harishbabu/projects/PIPNet/runs/046-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=10per-desc_orth-on-rel"
# run_path = "/home/harishbabu/projects/PIPNet/runs/047-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc"
# run_path = "/home/harishbabu/projects/PIPNet/runs/048-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=5per-desc_tanh-desc_unit-sphere"
# run_path = "/home/harishbabu/projects/PIPNet/runs/051-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/052-CUB-18-imgnet_OOD_cnext26_img=224_nprotos=4per-desc_tanh-desc_unit-sphere_AW=5-TW=2-UW=2-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/055-CUB-18_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/056-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/057-CUB-18-imgnet_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"
# run_path = "/home/harishbabu/projects/PIPNet/runs/058-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2"

# with unit sphere
# run_path = "/home/harishbabu/projects/PIPNet/runs/059-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# unit sphere with softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/065-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# original hpipnet with 20 protos per node no KO, no OOD, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/062-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

# original hpipnet with 20 protos per node no KO, no OOD, WITH tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/063-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD_tanh-desc"

# with unit sphere but no AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/066-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere_finetune=5_no-meanpool_no-softmax_no-align_no-uni_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, with softmax, no tanh-desc, INCORRECT
# run_path = "/home/harishbabu/projects/PIPNet/runs/067-incorrect-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# with unit sphere, protopool, no softmax, no tanh-desc
# run_path = "/home/harishbabu/projects/PIPNet/runs/068-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_AW=3-TW=2-UW=3-CW=2_batch=20"

# 071 with bias
# run_path = "/home/harishbabu/projects/PIPNet/runs/071-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_with-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 072 gumbel softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/072-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 073 gumbel softmax, tau-1.0
# run_path = "/home/harishbabu/projects/PIPNet/runs/073-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-gumbel-softmax-tau=1_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 075 068 with focal loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/075-068-with-focal_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 076 cs followed by softmax. Uses align_pf along with align+uni
# run_path = "/home/harishbabu/projects/PIPNet/runs/076_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2-APW=5_batch=20"

# 074 multiply_cs_softmax
# run_path = "/home/harishbabu/projects/PIPNet/runs/074-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_unit-sphere-protopool_finetune=5_no-meanpool_with-softmax_multi-cs-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 077 unit sphere protopool with cosin no softmax constant 20 protos per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/077_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20-sphere-protopool_finetune=5_align-pf-during-training_no-meanpool_no-softmax_no-addon-bias_AW=3-TW=2-UW=3-CW=2_batch=20"

# 082 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/082-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 083 unit sphere cs followed by softmax with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/083-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-align_no-uni_no-mm-loss_batch=48"

# 085 unit sphere cs followed by softmax-with-tau with minmazimize loss
# run_path = "/home/harishbabu/projects/PIPNet/runs/085-notebook-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=12"

# 091 basic gaussian multiplier on stage 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/091-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 092 basic gaussian multiplier on stage 3, 4
# run_path = "/home/harishbabu/projects/PIPNet/runs/092-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=3,4|1.0|50_img=224_latent-dim=256_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 093 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/093-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 094 128 dim linear
# run_path = "/home/harishbabu/projects/PIPNet/runs/094-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_latent-dim=128_nprotos=20_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=1-CW=2_mm-loss_batch=48"

# 095 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/095-091-woALUNI-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_no-AL_no-UNI_mm-loss_batch=48"

# 096 ablation 091 without AL+UNI
# run_path = "/home/harishbabu/projects/PIPNet/runs/096-091-wfocal-CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 097 - 091 with bg
# run_path = "/home/harishbabu/projects/PIPNet/runs/097-091-wbg-CUB-18_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_finetune=5_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0100 cub29 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/100_CUB-29-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48"

# 0101 baseline with 4 per desc per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/101-baseline-CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=4per-desc_no-KO_no-OOD"

# 0103 091 with 20 per node
run_path = "/home/harishbabu/projects/PIPNet/runs/103-091-wProtoPool20PerNode_CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48"

# 098 091 without AL + UNI
# run_path = '/home/harishbabu/projects/PIPNet/runs/098-091-woALUNI_finetune=0_CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=4per-leaf-desc_unit-sphere_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_mm-loss_batch=48'

# 0107 091 with 20 per node
# run_path = "/home/harishbabu/projects/PIPNet/runs/107-baseline_LOU_CUB-18-imgnet_with-equalize-aug_cnext26_img=224_nprotos=20_no-KO_no-OOD"

try:
    sys.path.remove('/home/harishbabu/projects/PIPNet')
except:
    pass
sys.path.insert(0, os.path.join(run_path, 'source_clone'))


In [4]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

In [5]:
from PIL import Image
import numpy as np
import pdb

def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

def get_bb_gaussian_threshold(latent_activation, sigma=1.0, percentile=97, extend_h=0, extend_w=0):
    # latent_activation -> []
    upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, \
                                                                 image_size=(args.image_size, args.image_size))
    upscaled_similarity = minmaxscale(upscaled_similarity)
    upscaled_similarity = gaussian(upscaled_similarity, sigma=sigma)
    upscaled_similarity = threshold_local(upscaled_similarity, block_size=15, method='mean')
    h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity ,percentile=97)
    h_min = max(0, h_min-extend_h)
    h_max = min(upscaled_similarity.shape[0], h_max+extend_h)
    w_min = max(0, w_min-extend_w)
    w_max = min(upscaled_similarity.shape[1], w_max+extend_w)
    return h_min, h_max, w_min, w_max


def minmaxscale(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())

from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

## Load Model

In [6]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('/home/harishbabu/data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

exp_no = int(os.path.basename(run_path)[:3])

if exp_no < 77:
    if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
        for node in root.nodes_with_children():
            node.set_num_protos(args.num_protos_per_descendant)
else:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))

------------------------- No discretization -------------------------


In [7]:
args.batch_size = 1

print(args.batch_size)

1


In [8]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

# args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
# args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)

print(args.batch_size, trainloader.batch_size)

if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
print(net.eval())
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
# with torch.no_grad():
#     xs1, _, _ = next(iter(trainloader))
#     xs1 = xs1.to(device)
#     proto_features, _, _ = net(xs1)
#     wshape = proto_features['root'].shape[-1]
#     args.wshape = wshape #needed for calculating image patch size
#     print("Output shape: ", proto_features['root'].shape, flush=True)
    
args.wshape = 26

Dropping 0 samples from trainloader
Dropping 0 samples from trainloader_normal
Dropping 0 samples from trainloader_normal_augment
Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
1 1
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}
stage 4
Number of prototypes:  20
----------Prototypes per descendant: 0----------
Assigned 20 protos to node root
Assign

In [9]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.__version__)

True
11.7
2.0.0+cu117


In [10]:
args.basic_cnext_gaussian_multiplier

'4|1.0|50'

# Proto activations on leaf descendents - topk images using either NAIVE-HPIPNET or UNIT-SPACE-PROTOPOOL with HEATMAP (CANON)

In [11]:
# Proto activations on leaf descendents - topk images

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision
import torch.nn.functional as F



def find_top_percentile_bbox(image, percentile=95):
    threshold = np.percentile(image.flatten(), percentile)
    mask = image >= threshold
    coords = np.argwhere(mask)
    if coords.size == 0:
        return None, None, None, None
    h_min, w_min = coords.min(axis=0)
    h_max, w_max = coords.max(axis=0)
    h_min, h_max, w_min, w_max = map(int, [h_min, h_max, w_min, w_max])
    return h_min, h_max, w_min, w_max

def find_high_activation_crop(activation_map, percentile=95):
    threshold = np.percentile(activation_map, percentile)
    mask = np.ones(activation_map.shape)
    mask[activation_map < threshold] = 0
    lower_y, upper_y, lower_x, upper_x = 0, 0, 0, 0
    for i in range(mask.shape[0]):
        if np.amax(mask[i]) > 0.5:
            lower_y = i
            break
    for i in reversed(range(mask.shape[0])):
        if np.amax(mask[i]) > 0.5:
            upper_y = i
            break
    for j in range(mask.shape[1]):
        if np.amax(mask[:,j]) > 0.5:
            lower_x = j
            break
    for j in reversed(range(mask.shape[1])):
        if np.amax(mask[:,j]) > 0.5:
            upper_x = j
            break
    return lower_y, upper_y+1, lower_x, upper_x+1

def get_upscaled_activation_uninterpolated(latent_activation, image_size):
    image_a = latent_activation.cpu().numpy()
    min_image_a = image_a.min()
    max_image_a = image_a.max()
    image_a = (image_a - min_image_a) / (max_image_a - min_image_a)
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((image_size[-1], \
                                                                                            image_size[-2]), \
                                                                                          resample=Image.NEAREST ))
    reshaped_image_a = (reshaped_image_a / 255).astype('float16')
    reshaped_image_a = (reshaped_image_a * (max_image_a - min_image_a)) + min_image_a
    return reshaped_image_a

# added for NUMPY SAVING
def get_upscaled_activation_interpolated(latent_activation, image_size):
    image_a = latent_activation.cpu().numpy()
    min_image_a = image_a.min()
    max_image_a = image_a.max()
    image_a = (image_a - min_image_a) / (max_image_a - min_image_a)
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((image_size[-1], \
                                                                                            image_size[-2])))    
    reshaped_image_a = (reshaped_image_a / 255).astype('float16')
    reshaped_image_a = (reshaped_image_a * (max_image_a - min_image_a)) + min_image_a
    return reshaped_image_a

def functional_UnitConv2D(in_features, weight, bias, stride = 1, padding=0):
    normalized_weight = F.normalize(weight.data, p=2, dim=(1, 2, 3)) # Normalize the kernels to unit vectors
    normalized_input = F.normalize(in_features, p=2, dim=1) # Normalize the input to unit vectors
    if bias is not None:
        normalized_bias = F.normalize(bias.data, p=2, dim=0) # Normalize the kernels to unit vectors
    else:
        normalized_bias = None
    return F.conv2d(normalized_input, normalized_weight, normalized_bias, stride=stride, padding=padding)

def findCorrespondingToMax(base, target):
    output, indices = F.max_pool2d(base, kernel_size=(26, 26), return_indices=True)# these are logits
    tensor_flattened = target.view(target.shape[0], target.shape[1], -1)
    indices_flattened = indices.view(target.shape[0], target.shape[1], -1)
    corresponding_values_in_target = torch.gather(tensor_flattened, 2, indices_flattened)
    corresponding_values_in_target = corresponding_values_in_target.view(target.shape[0],\
                                     target.shape[1], 1, 1)
    pooled_target = corresponding_values_in_target
    return pooled_target

def customForwardWithCSandSoftmax(net, xs,  inference=False):
    features = net.module._net(xs) 
    proto_features = {}
    proto_features_cs = {}
    proto_features_softmaxed = {}
    pooled = {}
    pooled_cs = {}
    pooled_softmaxed = {}
    out = {}
    for node in net.module.root.nodes_with_children():
        # this may or may not be cosine similarity based on UniConv2D or Conv2d
        proto_features[node.name] = getattr(net.module, '_'+node.name+'_add_on')(features)
        
        #calculating cosine similarity
        prototypes = getattr(net.module, '_'+node.name+'_add_on')
        proto_features_cs[node.name] = functional_UnitConv2D(features, prototypes.weight, prototypes.bias)

        if net.module.args.softmax == 'y':
            softmax_tau = 0.2
            proto_features[node.name] = proto_features[node.name] / softmax_tau
            proto_features_softmaxed[node.name] = net.module._softmax(proto_features[node.name])
            proto_features[node.name] = proto_features_softmaxed[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'
        elif net.module.args.gumbel_softmax == 'y':
            proto_features_softmaxed[node.name] = net.module._gumbel_softmax(proto_features[node.name])
            proto_features[node.name] = proto_features_softmaxed[node.name] # will be overwritten if args.multiply_cs_softmax == 'y'

        if net.module.args.multiply_cs_softmax == 'y':
            proto_features[node.name] = proto_features_cs[node.name] * proto_features_softmaxed[node.name]
        pooled[node.name] = net.module._pool(proto_features[node.name])
        
        # this could be softmax or cosine similarity
        pooled_cs[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_cs[node.name])
        
        pooled_softmaxed[node.name] = findCorrespondingToMax(base=proto_features[node.name], \
                                                     target=proto_features_softmaxed[node.name])

        if inference:
            pooled[node.name] = torch.where(pooled[node.name] < 0.1, 0., pooled[node.name])  #during inference, ignore all prototypes that have 0.1 similarity or lower
        out[node.name] = getattr(net.module, '_'+node.name+'_classification')(pooled[node.name]) #shape (bs*2, num_classes) # these are logits

    return features, proto_features, pooled, pooled_cs, pooled_softmaxed, out

find_non_descendants = False # True, False # param
vizloader_name = 'projectloader'
bbox_percentile = 97
topk = 6 # param, args param
save_images = False #True
# added for NUMPY SAVING
save_activation_as_npy_path = 'activation_as_npy'
analysis_mode = False

font = ImageFont.truetype("arial.ttf", 50)
font2 = ImageFont.truetype("arial.ttf", 20)
font3 = ImageFont.truetype("arial.ttf", 30)

from datetime import datetime
txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
txt_file.write('\n')

def write_num_proto_details(proto_mean_activations, node_name, net, threshold, txt_file, args):
    
    rand_input = torch.randn((1, 3, args.image_size, args.image_size))
    with torch.no_grad():
        *_, pooled, out = net(rand_input)
    num_protos = pooled[node_name].shape[1]
    used_protos = len(proto_mean_activations)
    non_overspecific = 0
    for p in proto_mean_activations:
        logstr = '\t'*2 + f'Proto:{p} '
        protos_mean_for_all_leaf_descedants = []
        for leaf_descendent in proto_mean_activations[p]:
            mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
            protos_mean_for_all_leaf_descedants.append(mean_activation)
            
        if all([(mean_activation>0.2) for mean_activation in protos_mean_for_all_leaf_descedants]):
            non_overspecific += 1
            
    txt_file.write(f"Node:{node_name},Total:{num_protos},Used:{used_protos},Good:{non_overspecific},threshold={threshold}\n")


def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)



vizloader_dict = {'trainloader': trainloader,
                 'projectloader': projectloader,
                 'testloader': testloader,
                 'test_projectloader': test_projectloader}
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])


if type(vizloader_dict[vizloader_name].dataset) == ImageFolder:
    name2label = vizloader_dict[vizloader_name].dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
else:
    name2label = vizloader_dict[vizloader_name].dataset.dataset.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

#     name2label = projectloader.dataset.class_to_idx # param
#     label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(vizloader_dict[vizloader_name], node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        # change
#         if not find_non_descendants: 
#             # do only when finding descendants
#             if coarse_label2name[ys.item()] not in non_leaf_children_names:
#                 continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            model_output = customForwardWithCSandSoftmax(net, xs, inference=False)
            _, softmaxes, pooled, pooled_ip, pooled_softmax, _ = model_output
#             model_output = net(xs, inference=False)
#             if len(model_output) == 3:
#                 softmaxes, pooled, _ = model_output
#             elif len(model_output) == 4:
#                 _, softmaxes, pooled, _ = model_output
            pooled = pooled[node.name].squeeze(0)
            pooled_ip = pooled_ip[node.name].squeeze(0) 
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                # change
#                 if (len(relevant_proto_class_names) == 1):# and (relevant_proto_class_names[0] not in non_leaf_children_names):
#                     continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                if not find_non_descendants:
                    if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7] if analysis_mode else \
                                                label2name[orig_y.item()][4:]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                else:
                    if (coarse_label2name[ys.item()] not in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7] if analysis_mode else \
                                                label2name[orig_y.item()][4:]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), pooled_ip[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)
                
    write_num_proto_details(proto_mean_activations, node.name, net, threshold=0.2, txt_file=txt_file, args=args)
    
    print('Node', node.name)
    for class_label in range(classification_weights.shape[0]):
        child_name = (coarse_label2name[class_label])
        print('Num protos for', child_name, torch.nonzero(classification_weights[class_label, :] > 1e-3).shape[0])
        
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            
            logstr = '\t'*2 + f'Proto:{p} '
            mean_activation_of_every_leaf = []
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
                mean_activation_of_every_leaf.append(mean_activation)
            print(logstr)
            
            # if the mean_activation is less for all leaf descendants skip the node
            if all([mean_act < 0.2 for mean_act in mean_activation_of_every_leaf]):
                print(f'Skipping proto {p} of {node.name}')
                continue
            
            # have this for NON descendants
            if len(proto_mean_activations[p]) == 0:
                continue
            
            if save_images or save_activation_as_npy_path:
                patches = []
                right_descriptions = []
                text_region_width = 3 if analysis_mode else 2 # 3x the width of a patch
                for leaf_descendent, heap in proto_mean_activations[p].items():
                    heap = sorted(heap)[::-1]
                    mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    least_activation = min([round(activation, 2) for activation, *_ in proto_mean_activations[p][leaf_descendent]])
                    most_activation = max([round(activation, 2) for activation, *_ in proto_mean_activations[p][leaf_descendent]])
                    mean_cosine_similarity = round(np.mean([activation_inner_product for _, activation_inner_product, *_ in proto_mean_activations[p][leaf_descendent]]), 2)
                    # modified for NUMPY SAVING
                    for rank, ele in enumerate(heap):
                        activation, activation_inner_product, img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation = ele
                        image = transforms.Resize(size=(args.image_size, args.image_size))(Image.open(img_to_open))
                        img_tensor = transforms.ToTensor()(image)#.unsqueeze_(0) #shape (1, 3, h, w)
                        img_tensor_patch = img_tensor[:, h_coor_min:h_coor_max, w_coor_min:w_coor_max]
#                         overlayed_image_np = get_heatmap(latent_activation, img_tensor)
#                         overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
#                         patches.append(overlayed_image)
                        
                        overlayed_image_np = get_heatmap(latent_activation, img_tensor)
                        if analysis_mode:
                            overlayed_image_pil = Image.fromarray(overlayed_image_np)
                            draw = D.Draw(overlayed_image_pil)
                            text = f"{round(activation, 2), round(activation_inner_product, 2)}"
    #                         text_width, text_height = draw.textsize(text, font2)
                            bbox = draw.textbbox((0, 0), text, font2)
                            text_width = bbox[2] - bbox[0]
                            text_height = bbox[3] - bbox[1]
                            x, y = 224 - text_width - 5, 5  # 10 pixels padding from right
                            draw.text((x, y), text, font=font2, fill=(255, 255, 255))
                            overlayed_image_np = np.array(overlayed_image_pil)
                        
                        overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
                        
                        if analysis_mode:
                            upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, image_size=(args.image_size, args.image_size))
                            h_min, h_max, w_min, w_max = get_bb_gaussian_threshold(latent_activation, sigma=1.0, \
                                                                                   percentile=bbox_percentile, extend_h=0, extend_w=0)
                            bbox_coords = torch.tensor([[w_min, h_min, w_max, h_max]])
                            overlayed_image = torchvision.utils.draw_bounding_boxes((overlayed_image * 255).type(torch.uint8), \
                                                                                       bbox_coords, colors='red') / 255
                        
#                         plt_image = overlayed_bb_image.permute(1, 2, 0)# should be H, W, C with 0 to 1
#                         plt.imshow(plt_image)
#                         plt.show()
#                         pdb.set_trace()
                        patches.append(overlayed_image)
                        # added for NUMPY SAVING
                        if save_activation_as_npy_path:
                            upscaled_similarity_interpolated = get_upscaled_activation_interpolated(latent_activation,
                                                                                       image_size=(args.image_size, args.image_size))
                            latent_activation_npy = latent_activation.squeeze().cpu().numpy()
                            data = {'node_name': node.name,
                                    'proto_num': p,
                                    'leaf_desc': leaf_descendent,
                                     'rank': rank,
                                     'img_path': img_to_open,
                                     'img_filename': ntpath.basename(img_to_open),
                                     'activation': latent_activation_npy,
                                     'max_activation': activation}    
                            filename = str(rank)+ '-' + ntpath.basename(img_to_open) + '.npy'
                            save_path = os.path.join(run_path, save_activation_as_npy_path, \
                                                     node.name, str(p), leaf_descendent,
                                                     filename)
                            os.makedirs(os.path.dirname(save_path), exist_ok=True)
                            np.save(save_path, data, allow_pickle=True)

                    # description on the right hand side
                    text = f'{mean_activation}, {leaf_descendent}' if analysis_mode else \
                                f'{leaf_descendent}'
                    txtimage = Image.new("RGB", (patches[0].shape[-2]*text_region_width,patches[0].shape[-1]), (0, 0, 0))
                    draw = D.Draw(txtimage)
                    draw.text((200, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font3)
                    txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)
                    right_descriptions.append(txttensor)
                
                # weird thing padding should be zero for non descendants else it raises some error # change
                if find_non_descendants or (len(patches) == topk): # (len(patches) == topk) means there is only one leaf descendant
                    padding = 0
                else:
                    padding = 1

                grid = torchvision.utils.make_grid(patches, nrow=topk, padding=padding)
                grid_right_descriptions = torchvision.utils.make_grid(right_descriptions, nrow=1, padding=padding)

                # merging right description with the grid of images
                grid = torch.cat([grid_right_descriptions, grid], dim=-1)

                # description on the top
                text = f'Node:{node.name}, p{p}, Child:{child_classname}' if analysis_mode else \
                            f'Parent node:{node.name}, Child node:{child_classname}'
                txtimage = Image.new("RGB", (grid.shape[-1], 75), (0, 0, 0))
                draw = D.Draw(txtimage)
                draw.text((500, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)

                # merging top description with the grid of images
                grid = torch.cat([txttensor, grid], dim=1)
                
                if save_images:
                    prefix = 'non_' if find_non_descendants else ''
                    os.makedirs(os.path.join(run_path, prefix + f'descendent_specific_topk_heatmap_{vizloader_name}_{bbox_percentile}_ep={epoch}_analysis={analysis_mode}', node.name), exist_ok=True)
                    torchvision.utils.save_image(grid, os.path.join(run_path, prefix + f'descendent_specific_topk_heatmap_{vizloader_name}_{bbox_percentile}_ep={epoch}_analysis={analysis_mode}', node.name, f'{child_classname}-p{p}.png'))

txt_file.write('\n')
txt_file.close()
print('Done !!!')

Node root
Num protos for 052+053 5
Num protos for 004+086 3
	 Child: 052+053
		Proto:3 050_Eared_Grebe:(0.3851) 051_Horned_Grebe:(0.3841) 052_Pied_billed_Grebe:(0.382) 053_Western_Grebe:(0.3853) 
		Proto:6 050_Eared_Grebe:(0.7384) 051_Horned_Grebe:(0.7182) 052_Pied_billed_Grebe:(0.8137) 053_Western_Grebe:(0.6582) 
		Proto:8 050_Eared_Grebe:(0.3328) 051_Horned_Grebe:(0.3316) 052_Pied_billed_Grebe:(0.3187) 053_Western_Grebe:(0.3287) 
		Proto:11 050_Eared_Grebe:(0.1273) 051_Horned_Grebe:(0.1138) 052_Pied_billed_Grebe:(0.1784) 053_Western_Grebe:(0.0802) 
Skipping proto 11 of root
		Proto:17 050_Eared_Grebe:(0.0824) 051_Horned_Grebe:(0.0854) 052_Pied_billed_Grebe:(0.0854) 053_Western_Grebe:(0.0844) 
Skipping proto 17 of root
	 Child: 004+086
		Proto:4 001_Black_footed_Albatross:(0.765) 002_Laysan_Albatross:(0.7984) 003_Sooty_Albatross:(0.8077) 004_Groove_billed_Ani:(0.8352) 023_Brandt_Cormorant:(0.7957) 024_Red_faced_Cormorant:(0.8251) 025_Pelagic_Cormorant:(0.8196) 031_Black_billed_Cuckoo:

Node 052+053
Num protos for cub_052_Pied_billed_Grebe 4
Num protos for 053+050 2
	 Child: cub_052_Pied_billed_Grebe
		Proto:16 052_Pied_billed_Grebe:(0.0768) 
Skipping proto 16 of 052+053
		Proto:19 052_Pied_billed_Grebe:(0.525) 
		Proto:5 052_Pied_billed_Grebe:(0.0773) 
Skipping proto 5 of 052+053
		Proto:7 052_Pied_billed_Grebe:(0.8435) 
	 Child: 053+050
		Proto:12 050_Eared_Grebe:(0.0727) 051_Horned_Grebe:(0.0736) 053_Western_Grebe:(0.0725) 
Skipping proto 12 of 052+053
		Proto:6 050_Eared_Grebe:(0.8619) 051_Horned_Grebe:(0.8689) 053_Western_Grebe:(0.8663) 


Node 004+086
Num protos for 004+032 1
Num protos for 086+045 3
	 Child: 004+032
		Proto:0 004_Groove_billed_Ani:(0.0752) 031_Black_billed_Cuckoo:(0.072) 032_Mangrove_Cuckoo:(0.0714) 033_Yellow_billed_Cuckoo:(0.073) 
Skipping proto 0 of 004+086
	 Child: 086+045
		Proto:8 001_Black_footed_Albatross:(0.0765) 002_Laysan_Albatross:(0.0766) 003_Sooty_Albatross:(0.0716) 023_Brandt_Cormorant:(0.0779) 024_Red_faced_Cormorant:(0.0744) 025_Pelagic_Cormorant:(0.078) 045_Northern_Fulmar:(0.0807) 086_Pacific_Loon:(0.0825) 100_Brown_Pelican:(0.0727) 101_White_Pelican:(0.0714) 
Skipping proto 8 of 004+086
		Proto:18 001_Black_footed_Albatross:(0.0874) 002_Laysan_Albatross:(0.0845) 003_Sooty_Albatross:(0.0827) 023_Brandt_Cormorant:(0.0851) 024_Red_faced_Cormorant:(0.0794) 025_Pelagic_Cormorant:(0.0819) 045_Northern_Fulmar:(0.0858) 086_Pacific_Loon:(0.0823) 100_Brown_Pelican:(0.0811) 101_White_Pelican:(0.0823) 
Skipping proto 18 of 004+086
		Proto:19 001_Black_footed_Albatross:(0.0744) 002_Laysan_Albatr

Node 053+050
Num protos for cub_053_Western_Grebe 6
Num protos for 050+051 3
	 Child: cub_053_Western_Grebe
		Proto:5 053_Western_Grebe:(0.0961) 
Skipping proto 5 of 053+050
		Proto:9 053_Western_Grebe:(0.2298) 
		Proto:12 053_Western_Grebe:(0.0739) 
Skipping proto 12 of 053+050
		Proto:14 053_Western_Grebe:(0.0793) 
Skipping proto 14 of 053+050
		Proto:15 053_Western_Grebe:(0.8463) 
		Proto:19 053_Western_Grebe:(0.0756) 
Skipping proto 19 of 053+050
	 Child: 050+051
		Proto:10 050_Eared_Grebe:(0.0691) 051_Horned_Grebe:(0.0709) 
Skipping proto 10 of 053+050
		Proto:18 050_Eared_Grebe:(0.3031) 051_Horned_Grebe:(0.2972) 
		Proto:7 050_Eared_Grebe:(0.7479) 051_Horned_Grebe:(0.7773) 


Node 004+032
Num protos for cub_004_Groove_billed_Ani 2
Num protos for 032+033 2
	 Child: cub_004_Groove_billed_Ani
		Proto:19 004_Groove_billed_Ani:(0.0687) 
Skipping proto 19 of 004+032
		Proto:3 004_Groove_billed_Ani:(0.0763) 
Skipping proto 3 of 004+032
	 Child: 032+033
		Proto:13 031_Black_billed_Cuckoo:(0.8735) 032_Mangrove_Cuckoo:(0.8739) 033_Yellow_billed_Cuckoo:(0.8702) 
		Proto:5 031_Black_billed_Cuckoo:(0.0741) 032_Mangrove_Cuckoo:(0.0743) 033_Yellow_billed_Cuckoo:(0.075) 
Skipping proto 5 of 004+032


...

Node 045+101
Num protos for 045+003 1
Num protos for 101+023 1
	 Child: 045+003
		Proto:0 001_Black_footed_Albatross:(0.8629) 002_Laysan_Albatross:(0.8642) 003_Sooty_Albatross:(0.8601) 045_Northern_Fulmar:(0.8603) 
	 Child: 101+023
		Proto:12 023_Brandt_Cormorant:(0.8691) 024_Red_faced_Cormorant:(0.8735) 025_Pelagic_Cormorant:(0.8717) 100_Brown_Pelican:(0.8736) 101_White_Pelican:(0.8727) 


Node 045+003
Num protos for cub_045_Northern_Fulmar 16
Num protos for 003+002 2
	 Child: 003+002
		Proto:0 001_Black_footed_Albatross:(0.0695) 002_Laysan_Albatross:(0.073) 003_Sooty_Albatross:(0.0675) 
Skipping proto 0 of 045+003
		Proto:5 001_Black_footed_Albatross:(0.0717) 002_Laysan_Albatross:(0.0753) 003_Sooty_Albatross:(0.0708) 
Skipping proto 5 of 045+003
	 Child: cub_045_Northern_Fulmar
		Proto:1 045_Northern_Fulmar:(0.8093) 
		Proto:2 045_Northern_Fulmar:(0.0757) 
Skipping proto 2 of 045+003
		Proto:3 045_Northern_Fulmar:(0.0978) 
Skipping proto 3 of 045+003
		Proto:4 045_Northern_Fulmar:(0.0815) 
Skipping proto 4 of 045+003
		Proto:6 045_Northern_Fulmar:(0.0941) 
Skipping proto 6 of 045+003
		Proto:7 045_Northern_Fulmar:(0.107) 
Skipping proto 7 of 045+003
		Proto:8 045_Northern_Fulmar:(0.0758) 
Skipping proto 8 of 045+003
		Proto:9 045_Northern_Fulmar:(0.1076) 
Skipping proto 9 of 045+003
		Proto:10 045_Northern_Fulmar:(0.0896) 
Skipping proto 10 of 045+003
		Proto:11 045_Nor

Node 101+023
Num protos for 101+100 1
Num protos for 023+025 2
	 Child: 101+100
		Proto:1 100_Brown_Pelican:(0.8764) 101_White_Pelican:(0.876) 
	 Child: 023+025
		Proto:2 023_Brandt_Cormorant:(0.07) 024_Red_faced_Cormorant:(0.0751) 025_Pelagic_Cormorant:(0.0761) 
Skipping proto 2 of 101+023
		Proto:19 023_Brandt_Cormorant:(0.8491) 024_Red_faced_Cormorant:(0.8549) 025_Pelagic_Cormorant:(0.8456) 


Node 003+002
Num protos for cub_003_Sooty_Albatross 15
Num protos for 002+001 2
	 Child: cub_003_Sooty_Albatross
		Proto:0 003_Sooty_Albatross:(0.0826) 
Skipping proto 0 of 003+002
		Proto:3 003_Sooty_Albatross:(0.084) 
Skipping proto 3 of 003+002
		Proto:4 003_Sooty_Albatross:(0.0911) 
Skipping proto 4 of 003+002
		Proto:5 003_Sooty_Albatross:(0.0797) 
Skipping proto 5 of 003+002
		Proto:6 003_Sooty_Albatross:(0.0927) 
Skipping proto 6 of 003+002
		Proto:7 003_Sooty_Albatross:(0.0878) 
Skipping proto 7 of 003+002
		Proto:9 003_Sooty_Albatross:(0.0841) 
Skipping proto 9 of 003+002
		Proto:11 003_Sooty_Albatross:(0.0809) 
Skipping proto 11 of 003+002
		Proto:12 003_Sooty_Albatross:(0.0771) 
Skipping proto 12 of 003+002
		Proto:13 003_Sooty_Albatross:(0.2066) 
		Proto:14 003_Sooty_Albatross:(0.838) 
		Proto:15 003_Sooty_Albatross:(0.0775) 
Skipping proto 15 of 003+002
		Proto:16 003_Sooty_Albatross:(0.1164) 
Skipping proto 16 of 003+002
		Proto:18 003_Sooty_Albatross:(0.1101) 
Skipping p

Node 023+025
Num protos for cub_023_Brandt_Cormorant 16
Num protos for 025+024 3
	 Child: cub_023_Brandt_Cormorant
		Proto:0 023_Brandt_Cormorant:(0.3112) 
		Proto:2 023_Brandt_Cormorant:(0.0879) 
Skipping proto 2 of 023+025
		Proto:4 023_Brandt_Cormorant:(0.0943) 
Skipping proto 4 of 023+025
		Proto:5 023_Brandt_Cormorant:(0.0836) 
Skipping proto 5 of 023+025
		Proto:6 023_Brandt_Cormorant:(0.0819) 
Skipping proto 6 of 023+025
		Proto:7 023_Brandt_Cormorant:(0.0935) 
Skipping proto 7 of 023+025
		Proto:8 023_Brandt_Cormorant:(0.0795) 
Skipping proto 8 of 023+025
		Proto:9 023_Brandt_Cormorant:(0.0828) 
Skipping proto 9 of 023+025
		Proto:10 023_Brandt_Cormorant:(0.0818) 
Skipping proto 10 of 023+025
		Proto:11 023_Brandt_Cormorant:(0.0797) 
Skipping proto 11 of 023+025
		Proto:12 023_Brandt_Cormorant:(0.0772) 
Skipping proto 12 of 023+025
		Proto:13 023_Brandt_Cormorant:(0.0788) 
Skipping proto 13 of 023+025
		Proto:14 023_Brandt_Cormorant:(0.0765) 
Skipping proto 14 of 023+025
		Prot

In [13]:
def eval_prototypes_cub_parts(npy_path, parts_loc_path, parts_name_path, imgs_id_path, imgs_sizes_path, args):
    patchsize, _ = get_patch_size(args)
    imgresize = float(args.image_size)
    
    img_filename_to_id = dict()
    with open(imgs_id_path) as f:
        for line in f:
            id, path = line.split('\n')[0].split(' ')
            img_filename = ntpath.basename(path)
            img_filename_to_id[img_filename] = int(id)
            
    img_id_to_size = dict()
    with open(imgs_sizes_path) as f:
        for line in f:
            id, width, height = line.split('\n')[0].split(' ')
            img_id_to_size[int(id)] = (float(width), float(height))
            
    img_id_to_part_xy_vis = dict()
    with open(parts_loc_path) as f:
        for line in f:
            img, partid, x, y, vis = line.split('\n')[0].split(' ')
            img, partid = int(img), int(partid)
            width, height = img_id_to_size[img]
            x = float(x) / width
            y = float(y) / height
            if img not in img_id_to_part_xy_vis.keys():
                img_id_to_part_xy_vis[img]=dict()
            if vis == '1':
                img_id_to_part_xy_vis[img][partid]=(x,y)
                
    partid_to_pos_in_vector = {1: 0,
                               2: 1,
                               3: 2,
                               4: 3,
                               5: 4,
                               6: 5,
                               7: 6, # left eye
                               8: 7, # left leg
                               9: 8, # left wing
                               10: 9,
                               11: 6, # right eye
                               12: 7, # right leg
                               13: 8, # right wing
                               14: 10,
                               15: 11}
    
    all_part_activations = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
                
    for root_dir, dirs, files in os.walk(npy_path):
        for file in files:
            npy_filepath = os.path.join(root_dir, file)
            data = np.load(npy_filepath, allow_pickle=True)
            activation = data['activation']
            img_id = img_filename_to_id[data['img_filename']]
            part_activation = np.zeros(12)
            partids_list = list(range(1, 16))
            
            # if both left eye and right eye are visible
            if (7 in img_id_to_part_xy_vis[img_id]) and (11 in img_id_to_part_xy_vis[img_id]):
                part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][7]
                part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                activation_parta = activation[part_loc_y, part_loc_x]
                part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][11]
                part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                activation_partb = activation[part_loc_y, part_loc_x]
                part_activation[partid_to_pos_in_vector[7]] = (activation_parta + activation_partb) / 2.
                partids_list.remove(7)
                partids_list.remove(11)
            # if both left leg and right leg are visible
            if (8 in img_id_to_part_xy_vis[img_id]) and (12 in img_id_to_part_xy_vis[img_id]):
                part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][8]
                part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                activation_parta = activation[part_loc_y, part_loc_x]
                part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][12]
                part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                activation_partb = activation[part_loc_y, part_loc_x]
                part_activation[partid_to_pos_in_vector[8]] = (activation_parta + activation_partb) / 2.
                partids_list.remove(8)
                partids_list.remove(12)
            # if both left wing and right wing are visible
            if (9 in img_id_to_part_xy_vis[img_id]) and (13 in img_id_to_part_xy_vis[img_id]):
                part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][9]
                part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                activation_parta = activation[part_loc_y, part_loc_x]
                part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][13]
                part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                activation_partb = activation[part_loc_y, part_loc_x]
                part_activation[partid_to_pos_in_vector[9]] = (activation_parta + activation_partb) / 2.
                partids_list.remove(9)
                partids_list.remove(13)
            
            for partid in partids_list:
                if partid in img_id_to_part_xy_vis:
                    part_loc_x, part_loc_y = img_id_to_part_xy_vis[img_id][partid]
                    part_loc_x, part_loc_y = int(part_loc_x * activation.shape[1]), int(part_loc_y * activation.shape[0])
                    part_activation[partid_to_pos_in_vector[partid]] = activation[part_loc_y, part_loc_x]
            
            all_part_activations[data['node_name']][data['proto_num']][data['leaf_desc']].append(part_activation)   
            
    for node_name

eval_prototypes_cub_parts(npy_path=os.path.join(run_path, 'activation_as_npy'), \
                          parts_loc_path='/projects/ml4science/harishbabu/data/CUB_200_2011/parts/part_locs.txt', \
                          parts_name_path='/projects/ml4science/harishbabu/data/CUB_200_2011/parts/parts.txt', \
                          imgs_id_path='/projects/ml4science/harishbabu/data/CUB_200_2011/images.txt', \
                          imgs_sizes_path='/projects/ml4science/harishbabu/data/CUB_200_2011/image_sizes.txt', \
                          args=args)


> /tmp/ipykernel_31177/857146827.py(48)eval_prototypes_cub_parts()
     46             pdb.set_trace()
     47 
---> 48             part_activation = np.zeros(15)
     49 
     50             for partid in range(1, 16):

ipdb> activation.max()
0.856
ipdb> activation.min()
0.008
ipdb> q


In [17]:
# for root, dirs, files in os.walk("/home/harishbabu/projects/PIPNet/runs/103-091-wProtoPool20PerNode_CUB-18-imgnet_with-equalize-aug_cnext26_BGM=4|1.0|50_img=224_nprotos=20_unit-sphere-protopool_no-meanpool_with-softmax-tau=0.2_no-addon-bias_AW=3-TW=2-MMW=2-UW=3-CW=2_batch=48/activation_as_npy", topdown=True):
#     print(root)
#     print(dirs)
#     print(files)
#     print('-*'*50)
    
# np.zeros(15)

all_part_activations = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

all_part_activations['node_name']['proto_num']['leaf_desc'].append(np.zeros(12))

In [18]:
all_part_activation

defaultdict(<function __main__.<lambda>()>,
            {'node_name': defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'proto_num': defaultdict(list,
                                      {'leaf_desc': [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]})})})

In [16]:
from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )

    return new_dataloader


vizloader_name = 'testloader'
vizloader_dict = {'trainloader': trainloader,
                 'projectloader': projectloader,
                 'testloader': testloader,
                 'test_projectloader': test_projectloader}

vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

print(vizloader_dict[vizloader_name].sampler)

print(vizloader_dict[vizloader_name].batch_sampler)

Dataset ImageFolder
    Number of datapoints: 844
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/test_crop
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [17]:
vizloader_name = 'testloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 844
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/test_crop
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [18]:
vizloader_name = 'test_projectloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 844
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/test_full
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [19]:
vizloader_name = 'projectloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 869
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
           )


In [22]:
vizloader_name = 'trainloader'
vizloader_dict[vizloader_name] = unshuffle_dataloader(vizloader_dict[vizloader_name])
print(vizloader_dict[vizloader_name].dataset)

Dataset ImageFolder
    Number of datapoints: 869
    Root location: /projects/ml4science/harishbabu/data/CUB_29_pipnet_224/dataset_segmented_imgnet_pt/train_crop


# Proto activations on leaf descendents - topk images using  NAIVE-HPIPNET with HEATMAP

In [9]:
# Proto activations on leaf descendents - topk images

def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

from util.data import ModifiedLabelLoader
from collections import defaultdict
import heapq
import pdb
from util.vis_pipnet import get_img_coordinates
import torchvision.transforms as transforms
from PIL import ImageFont, Image, ImageDraw as D
import torchvision
from datetime import datetime
txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
txt_file.write('\n')

find_non_descendants = False # True, False # param
topk = 6
save_images = False # True
font = ImageFont.truetype("arial.ttf", 50)
save_activation_as_npy_path = 'activation_as_npy'

from datetime import datetime
txt_file = open(os.path.join(run_path, "num_proto_details_"+datetime.now().strftime("%m:%d:%H:%M:%S")+".txt"), "a")
txt_file.write('\n')

def write_num_proto_details(proto_mean_activations, node_name, net, threshold, txt_file, args):
    
    rand_input = torch.randn((1, 3, args.image_size, args.image_size))
    with torch.no_grad():
        *_, pooled, out = net(rand_input)
    num_protos = pooled[node_name].shape[1]
    used_protos = len(proto_mean_activations)
    non_overspecific = 0
    for p in proto_mean_activations:
        logstr = '\t'*2 + f'Proto:{p} '
        protos_mean_for_all_leaf_descedants = []
        for leaf_descendent in proto_mean_activations[p]:
            mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
            protos_mean_for_all_leaf_descedants.append(mean_activation)
            
        if all([(mean_activation>0.2) for mean_activation in protos_mean_for_all_leaf_descedants]):
            non_overspecific += 1
            
    txt_file.write(f"Node:{node_name},Total:{num_protos},Used:{used_protos},Good:{non_overspecific},threshold={threshold}\n")


def get_heap():
    list_ = []
    heapq.heapify(list_)
    return list_

patchsize, skip = get_patch_size(args)

for node in root.nodes_with_children():
#     if node.name == 'root':
#         continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = projectloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(projectloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}
    
    imgs = modifiedLabelLoader.filtered_imgs

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> list of top-k activations
    proto_mean_activations = defaultdict(lambda: defaultdict(get_heap))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
#         if coarse_label2name[ys.item()] not in non_leaf_children_names:
#             continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            model_output = net(xs, inference=False)
            if len(model_output) == 3:
                softmaxes, pooled, _ = model_output
            elif len(model_output) == 4:
                _, softmaxes, pooled, _ = model_output
            pooled = pooled[node.name].squeeze(0) 
            softmaxes = softmaxes[node.name]#.squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]
                
                # Take the max per prototype.                             
                max_per_prototype, max_idx_per_prototype = torch.max(softmaxes, dim=0)
                max_per_prototype_h, max_idx_per_prototype_h = torch.max(max_per_prototype, dim=1)
                max_per_prototype_w, max_idx_per_prototype_w = torch.max(max_per_prototype_h, dim=1) #shape (num_prototypes)
                
                h_idx = max_idx_per_prototype_h[p, max_idx_per_prototype_w[p]]
                w_idx = max_idx_per_prototype_w[p]

                if len(relevant_proto_class_names) == 0:
                    continue
                
#                 if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
#                     continue
                
                h_coor_min, h_coor_max, w_coor_min, w_coor_max = get_img_coordinates(args.image_size, softmaxes.shape, patchsize, skip, h_idx, w_idx)
                latent_activation = softmaxes[:, p, :, :]
                
                if not find_non_descendants:
                    if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                else:
                    if (coarse_label2name[ys.item()] not in relevant_proto_class_names):
                        child_node = root.get_node(coarse_label2name[ys.item()])
                        leaf_descendent = label2name[orig_y.item()][4:7]
                        img_to_open = imgs[i][0] # it is a tuple of (path to image, lable)
                        if topk and (len(proto_mean_activations[p][leaf_descendent]) >= topk):
                            heapq.heappushpop(proto_mean_activations[p][leaf_descendent],\
                                              (pooled[p].item(), img_to_open,\
                                               (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))
                        else:
                            heapq.heappush(proto_mean_activations[p][leaf_descendent],\
                                           (pooled[p].item(), img_to_open,\
                                            (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation))

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

    write_num_proto_details(proto_mean_activations, node.name, net, threshold=0.2, txt_file=txt_file, args=args)

    print('Node', node.name)
    for child_classname in class_and_prototypes:
        
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            
            logstr = '\t'*2 + f'Proto:{p} '
            mean_activation_of_every_leaf = []
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                num_images = len(proto_mean_activations[p][leaf_descendent])
                logstr += f'{leaf_descendent}:({mean_activation}) '
                mean_activation_of_every_leaf.append(mean_activation)
            print(logstr)
            
            # if the mean_activation is less for all leaf descendants skip the node
            if all([mean_act < 0.2 for mean_act in mean_activation_of_every_leaf]):
                print(f'Skipping proto {p} of {node.name}')
                continue
            
            # have this for NON descendants
            if len(proto_mean_activations[p]) == 0:
                continue
            
            if save_images or save_activation_as_npy_path:
                patches = []
                right_descriptions = []
                text_region_width = 3 # 3x the width of a patch
                for leaf_descendent, heap in proto_mean_activations[p].items():
                    heap = sorted(heap)[::-1]
                    mean_activation = round(np.mean([activation for activation, *_ in proto_mean_activations[p][leaf_descendent]]), 4)
                    for rank, ele in enumerate(heap):
                        activation, img_to_open, (h_coor_min, h_coor_max, w_coor_min, w_coor_max), latent_activation = ele
                        image = transforms.Resize(size=(args.image_size, args.image_size))(Image.open(img_to_open))
                        img_tensor = transforms.ToTensor()(image)#.unsqueeze_(0) #shape (1, 3, h, w)
                        img_tensor_patch = img_tensor[:, h_coor_min:h_coor_max, w_coor_min:w_coor_max]
                        overlayed_image_np = get_heatmap(latent_activation, img_tensor)
                        overlayed_image = torch.tensor(overlayed_image_np).permute(2, 0, 1).float() / 255.
                        patches.append(overlayed_image)
                        
                        # added for NUMPY SAVING
                        
                        if save_activation_as_npy_path:
#                             upscaled_similarity_interpolated = get_upscaled_activation_interpolated(latent_activation,
#                                                                                        image_size=(args.image_size, args.image_size))
                            latent_activation_npy = latent_activation.squeeze().cpu().numpy()
                            data = {'node_name': node.name,
                                    'proto_num': p,
                                    'leaf_desc': leaf_descendent,
                                     'rank': rank,
                                     'img_path': img_to_open,
                                     'img_filename': ntpath.basename(img_to_open),
                                     'activation': latent_activation_npy,
                                     'max_activation': activation}    
                            filename = str(rank)+ '-' + ntpath.basename(img_to_open) + '.npy'
                            save_path = os.path.join(run_path, save_activation_as_npy_path, \
                                                     node.name, str(p), leaf_descendent,
                                                     filename)
                            os.makedirs(os.path.dirname(save_path), exist_ok=True)
                            np.save(save_path, data, allow_pickle=True)

                    # description on the right hand side
                    text = f'{mean_activation}, {leaf_descendent}'
                    txtimage = Image.new("RGB", (patches[0].shape[-2]*text_region_width,patches[0].shape[-1]), (0, 0, 0))
                    draw = D.Draw(txtimage)
                    draw.text((150, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                    txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)
                    right_descriptions.append(txttensor)
                    
                # weird thing padding should be zero for non descendants else it raises some error # change
                if find_non_descendants or (len(patches) == topk): # (len(patches) == topk) means there is only one leaf descendant
                    padding = 0
                else:
                    padding = 1

                grid = torchvision.utils.make_grid(patches, nrow=topk, padding=padding)
                grid_right_descriptions = torchvision.utils.make_grid(right_descriptions, nrow=1, padding=padding)

                # merging right description with the grid of images
                grid = torch.cat([grid, grid_right_descriptions], dim=-1)

                # description on the top
                text = f'Node:{node.name}, p{p}, Child:{child_classname}'
                txtimage = Image.new("RGB", (grid.shape[-1], args.wshape), (0, 0, 0))
                draw = D.Draw(txtimage)
                draw.text((150, patches[0].shape[1]//2), text, anchor='mm', fill="white", font=font)
                txttensor = transforms.ToTensor()(txtimage)#.unsqueeze_(0)

                # merging top description with the grid of images
                grid = torch.cat([grid, txttensor], dim=1)
                
                if save_images:
                    prefix = 'non_' if find_non_descendants else ''
                    os.makedirs(os.path.join(run_path, prefix+f'descendent_specific_topk_heatmap_ep={epoch}', node.name), exist_ok=True)
                    torchvision.utils.save_image(grid, os.path.join(run_path, prefix+f'descendent_specific_topk_heatmap_ep={epoch}', node.name, f'{child_classname}-p{p}.png'))

txt_file.write('\n')
txt_file.close()
print('Done !!!')

Node root
	 Child: 004+086
		Proto:0 001:(0.4387) 002:(0.9556) 003:(0.6655) 004:(0.17) 023:(0.5825) 024:(0.2604) 025:(0.4885) 031:(0.9433) 032:(0.9985) 033:(0.9995) 045:(0.7229) 086:(0.7392) 100:(0.0796) 101:(0.1876) 
		Proto:1 001:(0.9697) 002:(0.9837) 003:(0.9768) 004:(0.893) 023:(0.8102) 024:(0.6288) 025:(0.8527) 031:(0.1947) 032:(0.2876) 033:(0.1525) 045:(0.5682) 086:(0.4189) 100:(0.9993) 101:(0.9893) 
		Proto:2 001:(0.2888) 002:(0.0461) 003:(0.2407) 004:(0.2767) 023:(0.9861) 024:(0.9961) 025:(0.9173) 031:(0.0071) 032:(0.2882) 033:(0.0346) 045:(0.0557) 086:(0.3827) 100:(0.9999) 101:(0.9983) 
		Proto:4 001:(0.0727) 002:(0.0254) 003:(0.1299) 004:(0.0059) 023:(0.3014) 024:(0.0115) 025:(0.0497) 031:(0.9358) 032:(0.9471) 033:(0.4799) 045:(0.0294) 086:(0.2171) 100:(0.0101) 101:(0.0152) 
		Proto:5 001:(0.9988) 002:(0.9276) 003:(0.9996) 004:(1.0) 023:(0.9974) 024:(0.8148) 025:(0.9991) 031:(0.999) 032:(0.9981) 033:(0.9956) 045:(0.9928) 086:(1.0) 100:(0.9888) 101:(0.8972) 
		Proto:6 001:(0.0

Node 052+053
	 Child: cub_052_Pied_billed_Grebe
		Proto:0 052:(0.9587) 
		Proto:8 052:(0.9988) 
		Proto:4 052:(1.0) 
		Proto:14 052:(0.9999) 
	 Child: 053+050
		Proto:1 050:(0.5616) 051:(0.5862) 053:(0.9558) 
		Proto:2 050:(0.8769) 051:(0.9873) 053:(0.996) 
		Proto:3 050:(0.8863) 051:(0.9899) 053:(0.9941) 
		Proto:6 050:(0.9912) 051:(0.9956) 053:(0.9926) 
		Proto:7 050:(0.9099) 051:(0.9982) 053:(0.9785) 
		Proto:9 050:(0.9261) 051:(0.9979) 053:(0.9905) 
		Proto:10 050:(0.8899) 051:(0.8944) 053:(0.1667) 
		Proto:11 050:(0.9898) 051:(0.9986) 053:(0.9881) 
		Proto:12 050:(0.9816) 051:(0.8963) 053:(0.9559) 
		Proto:13 050:(0.9997) 051:(0.9986) 053:(0.9464) 
		Proto:16 050:(0.9464) 051:(0.9809) 053:(0.9977) 
		Proto:17 050:(0.4604) 051:(0.856) 053:(0.5544) 
		Proto:19 050:(0.985) 051:(0.9821) 053:(0.907) 


Node 004+086
	 Child: 004+032
		Proto:0 004:(0.1669) 031:(0.975) 032:(0.9976) 033:(0.9986) 
		Proto:2 004:(0.9407) 031:(0.8734) 032:(0.9983) 033:(0.9984) 
		Proto:6 004:(0.998) 031:(0.9861) 032:(0.9936) 033:(0.9978) 
		Proto:9 004:(0.0188) 031:(0.9905) 032:(0.9915) 033:(0.9419) 
		Proto:12 004:(0.9676) 031:(0.9251) 032:(0.1311) 033:(0.7084) 
		Proto:13 004:(0.9987) 031:(0.9993) 032:(0.9986) 033:(0.9878) 
		Proto:15 004:(0.9898) 031:(0.9665) 032:(0.9812) 033:(0.9096) 
	 Child: 086+045
		Proto:1 001:(0.9977) 002:(0.9994) 003:(0.9994) 023:(1.0) 024:(0.9995) 025:(1.0) 045:(0.9598) 086:(1.0) 100:(0.9997) 101:(0.9996) 
		Proto:3 001:(0.8767) 002:(0.4772) 003:(0.6463) 023:(0.0827) 024:(0.2068) 025:(0.4949) 045:(0.9821) 086:(0.8265) 100:(0.1141) 101:(0.1942) 
		Proto:4 001:(0.4335) 002:(0.6162) 003:(0.8708) 023:(0.0336) 024:(0.4325) 025:(0.8619) 045:(0.7866) 086:(0.4247) 100:(0.083) 101:(0.0924) 
		Proto:5 001:(0.0042) 002:(0.1048) 003:(0.1304) 023:(0.9997) 024:(1.0) 025:(0.9995) 045:(0.0253) 

Node 053+050
	 Child: 050+051
		Proto:2 050:(0.9991) 051:(0.9992) 
		Proto:3 050:(0.9961) 051:(0.9948) 
		Proto:5 050:(0.7436) 051:(0.9582) 
		Proto:8 050:(0.9991) 051:(0.9985) 
		Proto:9 050:(0.9825) 051:(0.9983) 
		Proto:12 050:(0.9828) 051:(0.9756) 
		Proto:13 050:(0.9684) 051:(0.9893) 
		Proto:15 050:(0.9449) 051:(0.9991) 
		Proto:17 050:(0.9658) 051:(0.9918) 
		Proto:19 050:(0.9853) 051:(0.987) 
	 Child: cub_053_Western_Grebe
		Proto:6 053:(0.9999) 
		Proto:7 053:(1.0) 
		Proto:10 053:(0.9995) 
		Proto:11 053:(1.0) 
		Proto:14 053:(0.9997) 
		Proto:18 053:(0.9986) 


Node 004+032
	 Child: 032+033
		Proto:0 031:(0.9871) 032:(0.9324) 033:(0.9727) 
		Proto:3 031:(0.9939) 032:(0.9712) 033:(0.9854) 
		Proto:4 031:(0.8933) 032:(0.9941) 033:(0.964) 
		Proto:6 031:(0.9911) 032:(0.9889) 033:(0.9346) 
		Proto:7 031:(0.9774) 032:(0.9868) 033:(0.9935) 
		Proto:8 031:(0.9818) 032:(1.0) 033:(0.9999) 
		Proto:9 031:(0.9739) 032:(0.9352) 033:(0.9807) 
		Proto:10 031:(0.9952) 032:(0.9684) 033:(0.9615) 
		Proto:12 031:(0.9998) 032:(0.9139) 033:(0.9824) 
		Proto:13 031:(0.996) 032:(0.961) 033:(0.7019) 
		Proto:14 031:(0.8839) 032:(0.9789) 033:(0.8816) 
		Proto:15 031:(0.9752) 032:(0.9963) 033:(0.9721) 
		Proto:17 031:(0.9634) 032:(1.0) 033:(1.0) 
		Proto:18 031:(0.9968) 032:(0.9823) 033:(0.9929) 
		Proto:19 031:(0.9951) 032:(0.9813) 033:(0.96) 
	 Child: cub_004_Groove_billed_Ani
		Proto:16 004:(1.0) 
		Proto:11 004:(0.9999) 
		Proto:5 004:(1.0) 


...

Node 045+101
	 Child: 101+023
		Proto:0 023:(0.9958) 024:(0.9998) 025:(0.993) 100:(0.477) 101:(0.4447) 
		Proto:1 023:(0.9977) 024:(0.9992) 025:(0.9984) 100:(0.9949) 101:(0.9965) 
		Proto:2 023:(0.9773) 024:(0.8123) 025:(0.9909) 100:(0.2911) 101:(0.2684) 
		Proto:6 023:(0.9963) 024:(0.9856) 025:(0.9607) 100:(0.9963) 101:(0.9967) 
		Proto:7 023:(0.8308) 024:(0.3724) 025:(0.9005) 100:(0.9921) 101:(0.9997) 
		Proto:8 023:(0.9999) 024:(0.9971) 025:(0.9997) 100:(0.9903) 101:(0.9991) 
		Proto:11 023:(0.8017) 024:(0.9776) 025:(0.9335) 100:(0.2619) 101:(0.3088) 
		Proto:19 023:(0.5875) 024:(0.9998) 025:(0.4644) 100:(0.9831) 101:(0.9972) 
	 Child: 045+003
		Proto:3 001:(0.9956) 002:(0.9531) 003:(0.9992) 045:(0.9467) 
		Proto:4 001:(0.9858) 002:(0.9971) 003:(0.8195) 045:(0.9777) 
		Proto:5 001:(0.9769) 002:(0.9968) 003:(0.9962) 045:(0.862) 
		Proto:9 001:(0.8898) 002:(0.8932) 003:(0.8217) 045:(0.8897) 
		Proto:10 001:(0.9912) 002:(0.9879) 003:(0.9518) 045:(0.2664) 
		Proto:12 001:(0.9763) 002:(0

Node 045+003
	 Child: 003+002
		Proto:0 001:(0.6931) 002:(0.9226) 003:(0.9288) 
		Proto:1 001:(0.9957) 002:(0.9566) 003:(0.9306) 
		Proto:2 001:(0.9989) 002:(0.9971) 003:(0.9987) 
		Proto:3 001:(0.9956) 002:(0.9914) 003:(0.9962) 
		Proto:4 001:(0.8615) 002:(0.9973) 003:(0.8051) 
		Proto:9 001:(0.9994) 002:(0.9418) 003:(0.9977) 
		Proto:10 001:(0.9208) 002:(0.8984) 003:(0.953) 
		Proto:11 001:(0.9848) 002:(0.9961) 003:(0.9482) 
		Proto:15 001:(0.9561) 002:(0.8994) 003:(0.9949) 
		Proto:16 001:(0.9867) 002:(0.9968) 003:(0.9275) 
		Proto:17 001:(0.9679) 002:(0.9996) 003:(0.5497) 
		Proto:18 001:(0.9951) 002:(0.9953) 003:(0.9935) 
	 Child: cub_045_Northern_Fulmar
		Proto:8 045:(0.9968) 
		Proto:14 045:(0.9997) 
		Proto:6 045:(0.9956) 
		Proto:7 045:(0.9956) 


Node 101+023
	 Child: 023+025
		Proto:0 023:(0.9981) 024:(0.9966) 025:(0.9942) 
		Proto:2 023:(0.9889) 024:(0.9998) 025:(0.9873) 
		Proto:4 023:(0.9947) 024:(0.9999) 025:(0.9994) 
		Proto:6 023:(0.9881) 024:(0.9915) 025:(0.9961) 
		Proto:9 023:(0.9338) 024:(0.9916) 025:(0.9786) 
		Proto:10 023:(0.9993) 024:(0.9931) 025:(0.9978) 
		Proto:11 023:(0.9996) 024:(0.9992) 025:(0.9999) 
		Proto:18 023:(0.9862) 024:(0.9966) 025:(0.9983) 
	 Child: 101+100
		Proto:1 100:(0.9975) 101:(0.8373) 
		Proto:3 100:(0.9997) 101:(0.9995) 
		Proto:7 100:(0.9993) 101:(0.997) 
		Proto:8 100:(0.9996) 101:(0.9997) 
		Proto:13 100:(0.9981) 101:(0.9569) 
		Proto:15 100:(0.9939) 101:(0.9951) 
		Proto:16 100:(1.0) 101:(1.0) 


Node 003+002
	 Child: 002+001
		Proto:0 001:(0.9971) 002:(0.9996) 
		Proto:3 001:(1.0) 002:(1.0) 
		Proto:9 001:(0.9954) 002:(0.9964) 
		Proto:11 001:(0.9777) 002:(0.9965) 
		Proto:12 001:(0.9846) 002:(0.9996) 
		Proto:13 001:(0.9986) 002:(0.9993) 
		Proto:15 001:(0.8837) 002:(0.9471) 
		Proto:17 001:(0.9994) 002:(0.9926) 
		Proto:18 001:(0.9971) 002:(0.9977) 
	 Child: cub_003_Sooty_Albatross
		Proto:8 003:(0.9986) 
		Proto:1 003:(0.9999) 
		Proto:5 003:(1.0) 
		Proto:7 003:(0.9998) 


Node 023+025
	 Child: cub_023_Brandt_Cormorant
		Proto:0 023:(1.0) 
		Proto:4 023:(0.9696) 
		Proto:7 023:(0.9994) 
		Proto:17 023:(0.9725) 
		Proto:18 023:(0.9999) 
	 Child: 025+024
		Proto:1 024:(0.9926) 025:(0.9678) 
		Proto:3 024:(0.9999) 025:(0.9961) 
		Proto:5 024:(0.9802) 025:(0.9883) 
		Proto:6 024:(0.9909) 025:(0.9704) 
		Proto:10 024:(0.9337) 025:(0.9344) 
		Proto:12 024:(1.0) 025:(0.9991) 
		Proto:13 024:(0.9999) 025:(0.9955) 
		Proto:14 024:(0.9993) 025:(0.9961) 
Done !!!
